In [28]:
#
# required to connect  
#
import mysql.connector
import os
connection = mysql.connector.connect(
  host=os.environ['MYSQL_IP'],
  user=os.environ['MYSQL_USER'],
  password=os.environ['MYSQL_PASSWORD'],
  database=os.environ['MYSQL_DATABASE'],
  port=int(os.environ['MYSQL_PORT'])
)

def push_panda_table_sql(connection, table_name, df):
    
    mycursor=connection.cursor()
    mycursor.execute("SHOW TABLES LIKE '"+table_name+"'")
    result = mycursor.fetchone()
    if not result:
        cols = "`,`".join([str(i) for i in df.columns.tolist()])
        db_to_crete = "CREATE TABLE `"+table_name+"` ("+' '.join(["`"+x+"` REAL," for x in df.columns.tolist()])[:-1]+")"
        print ("[Table {:s} created into SQL Server]".format(table_name))
        mycursor = connection.cursor()
        mycursor.execute(db_to_crete)

    cols = "`,`".join([str(i) for i in df.columns.tolist()])

    for i,row in df.iterrows():
        sql = "INSERT INTO `"+table_name+"` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        mycursor.execute(sql, tuple(row.astype(str)))
        connection.commit()

    mycursor.close()


def update_sql_value(connection, table_name, row_element, row_element_condition, 
                     colum_element, value, verbose=False):

    sql = "UPDATE `"+table_name+"` SET `"+colum_element+"` = "+ \
    str(value)+" WHERE `"+row_element+"` = "+str(row_element_condition)+";"
    if verbose: print(sql)
    try:
        mycursor = connection.cursor()
        mycursor.execute(sql)
        connection.commit()
        if verbose: print(mycursor.rowcount, "Update done")
        mycursor.close()
        return 0
    except:
        return 1
    
# update_sql_value(connection, table_name="Runlog", 
#                  row_element="run_number", 
#                  row_element_condition=268, 
#                  colum_element="storage_local_status", 
#                  value=4, verbose=True)
def update_runlog_replica_status(run_number, storage, status=-1, verbose=False):
    if storage=="local":
        storage="storage_local_status"
    elif storage=="cloud":
        storage="storage_cloud_status"
    elif storage=="tape":
        storage="storage_tape_status"
    else:
        return 1
    if verbose: print("Storage: "+storage)
    return update_sql_value(connection, table_name="Runlog", row_element="run_number", 
                     row_element_condition=run_number, 
                     colum_element=storage, value=status, 
                     verbose=verbose)
     
    

In [32]:
update_runlog_replica_status(run_number=268, storage="local", status=-1, verbose=False)

0